# Risk Flow Matrix Modeling

In [1]:
import pandas as pd

In [6]:
tracts = pd.read_csv('tracts_3.csv')

,Unnamed: 0,GEOID,ZIP_CODES,TRIPS_ORIG,TRIPS_DEST,POPULATION,STORES,CBSA_EMP,CBSA_POP,CBSA_WRK,...,No. Post Offices,No. Airports,No. Populated Places,No. Stores,Total Points of Interest,POP_DENSITY,NUM_WORKERS,EMPLOYMENT_DENSITY,PREDICTED_ORIG_TRIPS,PREDICTED_DEST_TRIPS
0,0,17091011700,0,27.0,111.0,3417,0,43299.0,113449.0,46799.0,...,0.0,1.0,0.0,0,8.0,7.059873,46799.0,2.442126,126.347,125.582
1,1,17091011800,0,93.0,141.0,2627,0,43299.0,113449.0,46799.0,...,0.0,0.0,1.0,0,3.0,5.838794,46799.0,1.327268,116.855,155.451
2,2,17119400951,0,433.0,299.0,4966,0,1261547.0,2812896.0,1237055.0,...,0.0,0.0,0.0,0,4.0,3.772428,1237055.0,0.566353,314.676,283.279
3,3,17119400952,0,4.0,155.0,3335,0,1261547.0,2812896.0,1237055.0,...,0.0,0.0,1.0,0,2.0,2.777786,1237055.0,0.242828,60.534,203.733
4,4,17135957500,['62533' '62560'],34.0,45.0,3273,0,0.0,0.0,0.0,...,0.0,1.0,7.0,0,49.0,0.128183,0.0,0.040990,41.602,70.198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3118,3118,17037000100,['60135'],145.0,245.0,6712,0,4161510.0,9461105.0,4066635.0,...,0.0,2.0,2.0,0,18.0,1.022809,4066635.0,0.274031,160.356,242.515
3119,3119,17037001500,0,346.0,156.0,3807,0,4161510.0,9461105.0,4066635.0,...,0.0,1.0,1.0,0,8.0,5.213170,4066635.0,1.535521,404.125,199.039
3120,3120,17037000400,['60178'],13.0,499.0,8622,2,4161510.0,9461105.0,4066635.0,...,0.0,4.0,4.0,2,24.0,0.252156,4066635.0,0.009529,193.709,440.224
3121,3121,17037000300,['60111' '60150'],110.0,59.0,2680,0,4161510.0,9461105.0,4066635.0,...,2.0,4.0,3.0,0,33.0,0.037200,4066635.0,0.012366,87.904,65.580


Group census tracts into counties that share the same FIPS code

In [55]:
geoids = tracts.loc[:,'GEOID'].astype(str).apply(lambda x : x[:5])
county_ids = pd.DataFrame(geoids)
county_groups = county_ids.groupby('GEOID', as_index = False)
county_df = pd.DataFrame(county_groups)
county_df.columns = ['FIPS','census_tracts']
county_df.loc[:,'census_tracts'] = county_df.loc[:,'census_tracts'].apply(lambda x : len(x))
county_df

,FIPS,census_tracts
0,17001,18
1,17003,4
2,17005,4
3,17007,7
4,17009,2
...,...,...
97,17195,18
98,17197,152
99,17199,15
100,17201,77


In [4]:
cases = pd.read_csv('counties_with_cases.csv')
cases

,FIPS,COUNTY,CASES
0,17003,Alexander,37.0
1,17005,Bond,68.0
2,17007,Boone,770.0
3,17009,Brown,15.0
4,17011,Bureau,247.0
...,...,...,...
96,17195,Whiteside,368.0
97,17197,Will,9576.0
98,17199,Williamson,477.0
99,17201,Winnebago,3814.0


In [52]:
cases.FIPS = cases.FIPS.astype(str)

In [54]:
cases.iloc[0,0]

'17003'

Merge the number of census tracts with each corresponding county

In [56]:
cases = cases.merge(county_df, on = 'FIPS')
cases

,FIPS,COUNTY,CASES,census_tracts
0,17003,Alexander,37.0,4
1,17005,Bond,68.0,4
2,17007,Boone,770.0,7
3,17009,Brown,15.0,2
4,17011,Bureau,247.0,10
...,...,...,...,...
96,17195,Whiteside,368.0,18
97,17197,Will,9576.0,152
98,17199,Williamson,477.0,15
99,17201,Winnebago,3814.0,77


In [57]:
cases.to_csv('counties_census_tracts_cases.csv')

Get the average cases per census tract for each census tract for a particular county

In [58]:
cases['avg_cases_per_tract'] = cases['CASES'] / cases['census_tracts']

In [59]:
cases

,FIPS,COUNTY,CASES,census_tracts,avg_cases_per_tract
0,17003,Alexander,37.0,4,9.250000
1,17005,Bond,68.0,4,17.000000
2,17007,Boone,770.0,7,110.000000
3,17009,Brown,15.0,2,7.500000
4,17011,Bureau,247.0,10,24.700000
...,...,...,...,...,...
96,17195,Whiteside,368.0,18,20.444444
97,17197,Will,9576.0,152,63.000000
98,17199,Williamson,477.0,15,31.800000
99,17201,Winnebago,3814.0,77,49.532468


In [73]:
fips_cases = cases.loc[:,('FIPS','avg_cases_per_tract')]
fips_cases.iloc[0,0]

'17003'

In [70]:
tracts.loc[:,'GEOID'] = tracts.loc[:,'GEOID'].astype(str)
tracts['FIPS'] = tracts['GEOID'].apply(lambda x : x[:5])

0       17091
1       17091
2       17119
3       17119
4       17135
        ...  
3118    17037
3119    17037
3120    17037
3121    17037
3122    17037
Name: FIPS, Length: 3123, dtype: object

In [75]:
tracts.FIPS.iloc[0]

'17091'

Join the data back to census tracts table

In [80]:
tracts = tracts.join(fips_cases.set_index('FIPS'), on = 'FIPS')

In [81]:
tracts

,Unnamed: 0,GEOID,ZIP_CODES,TRIPS_ORIG,TRIPS_DEST,POPULATION,STORES,CBSA_EMP,CBSA_POP,CBSA_WRK,...,No. Populated Places,No. Stores,Total Points of Interest,POP_DENSITY,NUM_WORKERS,EMPLOYMENT_DENSITY,PREDICTED_ORIG_TRIPS,PREDICTED_DEST_TRIPS,FIPS,avg_cases_per_tract
0,0,17091011700,0,27.0,111.0,3417,0,43299.0,113449.0,46799.0,...,0.0,0,8.0,7.059873,46799.0,2.442126,126.347,125.582,17091,63.068966
1,1,17091011800,0,93.0,141.0,2627,0,43299.0,113449.0,46799.0,...,1.0,0,3.0,5.838794,46799.0,1.327268,116.855,155.451,17091,63.068966
2,2,17119400951,0,433.0,299.0,4966,0,1261547.0,2812896.0,1237055.0,...,0.0,0,4.0,3.772428,1237055.0,0.566353,314.676,283.279,17119,46.803279
3,3,17119400952,0,4.0,155.0,3335,0,1261547.0,2812896.0,1237055.0,...,1.0,0,2.0,2.777786,1237055.0,0.242828,60.534,203.733,17119,46.803279
4,4,17135957500,['62533' '62560'],34.0,45.0,3273,0,0.0,0.0,0.0,...,7.0,0,49.0,0.128183,0.0,0.040990,41.602,70.198,17135,21.875000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3118,3118,17037000100,['60135'],145.0,245.0,6712,0,4161510.0,9461105.0,4066635.0,...,2.0,0,18.0,1.022809,4066635.0,0.274031,160.356,242.515,17037,45.714286
3119,3119,17037001500,0,346.0,156.0,3807,0,4161510.0,9461105.0,4066635.0,...,1.0,0,8.0,5.213170,4066635.0,1.535521,404.125,199.039,17037,45.714286
3120,3120,17037000400,['60178'],13.0,499.0,8622,2,4161510.0,9461105.0,4066635.0,...,4.0,2,24.0,0.252156,4066635.0,0.009529,193.709,440.224,17037,45.714286
3121,3121,17037000300,['60111' '60150'],110.0,59.0,2680,0,4161510.0,9461105.0,4066635.0,...,3.0,0,33.0,0.037200,4066635.0,0.012366,87.904,65.580,17037,45.714286


In [82]:
tracts.to_csv('tracts_4.csv')

In [1]:
import pandas as pd

Calculate the number of trips from census tract i to j for all i's and j's

In [2]:
OD = pd.read_csv('origin_dest_list.csv')

In [4]:
OD.columns = ["WORKPLACE","RESIDENCE","JOBS"]

In [7]:
OD = OD.loc[:,('RESIDENCE','WORKPLACE','JOBS')]

In [8]:
OD.loc[:,'RESIDENCE']

,RESIDENCE,WORKPLACE,JOBS
0,170010002022017,170010001001007,2
1,170010002011028,170010001001017,2
2,170010005001023,170010001001017,2
3,170010005002013,170010001001017,2
4,170010008003016,170010001001017,2
...,...,...,...
249702,171130059004003,172030307003299,2
249703,172030307001016,172030307003299,2
249704,172030307001021,172030307003299,2
249705,172030307003119,172030307003299,2


In [10]:
OD.loc[:,'RESIDENCE'] = OD.loc[:,'RESIDENCE'].astype(str).apply(lambda x : x[:11])
OD.loc[:,'WORKPLACE'] = OD.loc[:,'WORKPLACE'].astype(str).apply(lambda x : x[:11])
OD

,RESIDENCE,WORKPLACE,JOBS
0,17001000202,17001000100,2
1,17001000201,17001000100,2
2,17001000500,17001000100,2
3,17001000500,17001000100,2
4,17001000800,17001000100,2
...,...,...,...
249702,17113005900,17203030700,2
249703,17203030700,17203030700,2
249704,17203030700,17203030700,2
249705,17203030700,17203030700,2


In [11]:
OD['COMBINED'] = OD['RESIDENCE'] + OD['WORKPLACE']

In [21]:
OD = OD.loc[:,('RESIDENCE','WORKPLACE','COMBINED','JOBS')]
OD = OD.loc[:,('COMBINED','JOBS')]

In [24]:
OD

,COMBINED,JOBS
0,1700100020217001000100,2
1,1700100020117001000100,2
2,1700100050017001000100,2
3,1700100050017001000100,2
4,1700100080017001000100,2
...,...,...
249702,1711300590017203030700,2
249703,1720303070017203030700,2
249704,1720303070017203030700,2
249705,1720303070017203030700,2


In [27]:
OD2 = pd.DataFrame(OD.groupby('COMBINED', as_index = False).sum())

In [30]:
OD2['RESIDENCE'] = OD2['COMBINED'].apply(lambda x : x[:11])
OD2['WORKPLACE'] = OD2['COMBINED'].apply(lambda x : x[11:])
OD2 = OD2.drop(columns = 'COMBINED')

In [32]:
OD2 = OD2.loc[:,('RESIDENCE','WORKPLACE','JOBS')]

In [35]:
OD2.to_csv('origin_dest_trips_census_tract_level.csv', index = False)

In [37]:
tracts = pd.read_csv('tracts_4.csv')
tracts = tracts.drop(columns = ('Unnamed: 0'))

In [39]:
tracts = tracts.drop(columns = ('Unnamed: 0.1'))

In [41]:
tracts.to_csv('tracts_4.csv', index = False)

In [43]:
tracts.columns.values

array(['GEOID', 'ZIP_CODES', 'TRIPS_ORIG', 'TRIPS_DEST', 'POPULATION',
       'STORES', 'CBSA_EMP', 'CBSA_POP', 'CBSA_WRK', 'COUNTHU10',
       'TOTPOP10', 'HH', 'P_WRKAGE', 'AUTOOWN0', 'PCT_AO0', 'AUTOOWN1',
       'PCT_AO1', 'AUTOOWN2P', 'PCT_AO2P', 'WORKERS', 'R_LOWWAGEW',
       'R_MEDWAGEW', 'R_HIWAGEWK', 'R_PCTLOWWA', 'EMPTOT', 'E5_RET10',
       'E5_OFF10', 'E5_IND10', 'E5_SVC10', 'E5_ENT10', 'E8_RET10',
       'E8_OFF10', 'E8_IND10', 'E8_SVC10', 'E8_ENT10', 'E8_ED10',
       'E8_HLTH10', 'E8_PUB10', 'E_FEDT10', 'E_FEDRET10', 'E_FEDOFF10',
       'E_FEDIND10', 'E_FEDSVC10', 'E_FEDENT10', 'E_LOWWAGEW',
       'E_MEDWAGEW', 'E_HIWAGEWK', 'E_PCTLOWWA', 'AC_TOT', 'AC_WATER',
       'AC_LAND', 'AC_UNPR', 'D1A', 'D1B', 'D1C', 'D1C5_Ret10',
       'D1C5_Off10', 'D1C5_Ind10', 'D1C5_Svc10', 'D1C5_Ent10',
       'D1C8_Ret10', 'D1C8_Off10', 'D1C8_Ind10', 'D1C8_Svc10',
       'D1C8_Ent10', 'D1C8_Ed10', 'D1C8_Hlth1', 'D1C8_Pub10', 'D1D',
       'D1_flag', 'D2A_JPHH', 'D2B_E5MIX', 'D2B_E5MIXA

The list of commute trips from residence to workplace

In [53]:
OD2

,RESIDENCE,WORKPLACE,JOBS
0,17001000100,17001000100,139
1,17001000100,17001000201,12
2,17001000100,17001000202,9
3,17001000100,17001000400,10
4,17001000100,17001000500,119
...,...,...,...
80423,17203030700,17113005102,2
80424,17203030700,17143001600,5
80425,17203030700,17179021201,2
80426,17203030700,17203030601,6


Generate a unique set of origins in commute trips

In [52]:
origins = OD2.loc[:,'RESIDENCE'].unique().tolist()

Generate a doubly nested hashmap: \
first mapping: origin -> destination \
second mapping: destination -> number of trips

In [55]:
i = 0
mRes = {}
for origin in origins:
    mWork = {}
    while i < len(OD2) and OD2.iloc[i,0] == origin:
        workplace = OD2.iloc[i,1]
        jobs = OD2.iloc[i,2]
        mWork[workplace] = jobs
        i += 1
    mRes[origin] = mWork

In [113]:
tracts = tracts.rename(columns = {'avg_cases_per_tract' : 'CASES'})
tracts = tracts.loc[:,('GEOID','TRIPS_ORIG','CASES')]
tracts.loc[:,'GEOID'] = tracts.loc[:,'GEOID'].astype(str)
tracts.iloc[0,0]

'17091011700'

In [170]:
tracts

,GEOID,TRIPS_ORIG,CASES
0,17091011700,27.0,63.068966
1,17091011800,93.0,63.068966
2,17119400951,433.0,46.803279
3,17119400952,4.0,46.803279
4,17135957500,34.0,21.875000
...,...,...,...
3118,17037000100,145.0,45.714286
3119,17037001500,346.0,45.714286
3120,17037000400,13.0,45.714286
3121,17037000300,110.0,45.714286


Build risk flow model

Formula: \
risk from census tract from i to j: \
case_number_at_i * num_of_trips_from_i_to_j / sum_of_product_of_cases_at_j_and_flux_from_j_for_all_j's

In [173]:
tracts.loc[:,'TRIPS_ORIG'] = tracts.loc[:,'TRIPS_ORIG'].fillna(0)
tracts.loc[:,'CASES'] = tracts.loc[:,'CASES'].fillna(0)
total_risk_flow = sum([tracts.iloc[i, -1] * tracts.iloc[i, -2] for i in range(len(tracts))])
total_risk_flow

39555573.88410911

In [174]:
# populate horizontal header for matrix
import numpy as np
import math
matrix = []
# row = []
# row.append(0)
# h = tracts.loc[:,'GEOID'].values[:10].tolist() # horizontal header
# row = row + h
# matrix.append(row)

In [175]:
# populate rest of the rows
length = len(tracts)
for i in range(length):
    row = []
#     row.append(tracts.loc[:,'GEOID'].iloc[i]) # vertical header for matrix
    cases_at_i = tracts.iloc[i, -1]    # case number at i
    origin_at_i = tracts.iloc[i, 0] # residence
    m_work = mRes.get(origin_at_i, {})
    for j in range(length):       # for each dest at j
        dest_at_j = tracts.iloc[j, 0]    # geoid at dest j
        trips = 0
        if len(m_work) != 0:
            trips = m_work.get(dest_at_j, 0)
        row.append((cases_at_i * trips / total_risk_flow))
    matrix.append(row)

In [176]:
risk_flow_matrix = pd.DataFrame(matrix, index = tracts.loc[:,'GEOID'], columns = tracts.loc[:, 'GEOID'])

In [177]:
# export the matrix to an excel file
risk_flow_matrix.to_csv('risk_flow_matrix.csv')

0